In [ ]:
import pandas as pd
import numpy as np
import torch
from scipy.stats import gamma, norm
from darts.models import RNNModel,RegressionModel , RandomForest, XGBModel
import pywt
from darts import TimeSeries
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# Set random seed for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


Step 1: Load and Prepare Data
#Read and preprocess the input data


In [ ]:

def fit_gamma(precip: pd.Series) -> tuple:
    non_zero = precip[precip > 0]
    if len(non_zero) == 0:
        return np.nan, np.nan, 1.0
    shape, _, scale = gamma.fit(non_zero, floc=0)
    zero_prob = (precip == 0).mean()
    return shape, scale, zero_prob


def compute_spi(roll: pd.Series, shape: float, scale: float, zero_prob: float) -> np.ndarray:
    # probs = zero_prob + (1 - zero_prob) * gamma.cdf(roll, shape, scale=scale)
    probs = np.where(
    roll.isna(),
    np.nan,
    zero_prob + (1 - zero_prob) * gamma.cdf(roll, shape, scale=scale)
    )
    probs = np.clip(probs, 1e-10, 1 - 1e-10)
    return norm.ppf(probs)


def process_station(df_station: pd.DataFrame) -> pd.DataFrame:
    df_station = df_station.sort_values('ds').reset_index(drop=True)
    # shape, scale, zero_prob = fit_gamma(df_station['precip'])
    result = {'ds': df_station['ds']}

    for s in [1, 3, 6, 9, 12, 24]:
        roll = df_station['precip'].rolling(s, min_periods=s).sum()
        shape, scale, zero_prob = fit_gamma(roll.dropna())

        spi = compute_spi(roll, shape, scale, zero_prob)
        # z-score manually
        mean, std = np.nanmean(spi), np.nanstd(spi)
        result[f'SPI_{s}'] = (spi - mean) / std

    result_df = pd.DataFrame(result)
    result_df['station_id'] = df_station['station_id'].iloc[0]
    return result_df


# Load & aggregate to monthly totals
df = (
    pd.read_csv('../result/merged_data.csv', parse_dates=['data'])
      .assign(ds=lambda d: d['data'].dt.to_period('M').dt.to_timestamp())
      .groupby(['station_id', 'ds'])['rrr24']
      .sum()
      .reset_index(name='precip')
)

# Compute SPI for each station
spi_list = [process_station(g) for _, g in df.groupby('station_id')]
all_spi = pd.concat(spi_list, ignore_index=True)


class TaylorDiagram

In [ ]:

class TaylorDiagram:
    def __init__(self, ref_std, fig=None, rect=111, label='Reference'):
        self.ref_std = ref_std

        self.fig = fig if fig is not None else plt.figure()
        self.ax = self.fig.add_subplot(rect, polar=True)
        self.ax.set_theta_direction(-1)
        self.ax.set_theta_offset(np.pi / 2)

        self.sample_points = []

        # Set up axis
        self.ax.set_thetamin(0)
        self.ax.set_thetamax(90)
        self.ax.set_ylim(0, 1.5 * ref_std)

        # Set up correlation grid
        self._setup_axes()

        # Reference point
        self.ax.plot([0], [ref_std], 'k*', markersize=10, label=label)

    def _setup_axes(self):
        corrs = np.array([0.0, 0.2, 0.4, 0.6, 0.8, 0.9, 0.95, 0.99, 1.0])
        angles = np.arccos(corrs)
        self.ax.set_thetagrids(np.degrees(angles), labels=[f"{c:.2f}" for c in corrs])
        self.ax.set_ylabel('Standard Deviation')

    def add_sample(self, stddev, corrcoef, label, marker='o', c='C0'):
        theta = np.arccos(corrcoef)
        point = self.ax.plot(theta, stddev, marker, label=label, color=c)
        self.sample_points.append(point)

    def add_contours(self, levels=5, **kwargs):
        rs, ts = np.meshgrid(
            np.linspace(0, self.ax.get_ylim()[1]),
            np.linspace(0, np.pi / 2)
        )
        rms = np.sqrt(
            self.ref_std**2 + rs**2 - 2 * self.ref_std * rs * np.cos(ts)
        )
        contours = self.ax.contour(ts, rs, rms, levels=levels, **kwargs)
        return contours
    
    def plot_spi_trend(historical: TimeSeries, forecast: TimeSeries, title='SPI Trend'):
        full_series = historical.append(forecast)

        plt.figure(figsize=(12, 4))
        full_series.plot(label="SPI")
        plt.axvline(x=historical.end_time(), color='r', linestyle='--', label="Forecast Start")
        plt.title(title)
        plt.xlabel("Time")
        plt.ylabel("SPI")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()


Define Models

In [ ]:

class WBBLSTMModel:
    def __init__(self, wavelet='db1', level=1, **lstm_kwargs):
        self.wavelet = wavelet
        self.level = level
        self.lstm_model = RNNModel(model='LSTM', **lstm_kwargs)
        
    def wavelet_decompose(self, series: TimeSeries):
        coeffs = pywt.wavedec(series.values().flatten(), self.wavelet, level=self.level)
        return coeffs
    
    def wavelet_reconstruct(self, coeffs):
        return pywt.waverec(coeffs, self.wavelet)
    
    def fit(self, series: TimeSeries):
        # Decompose
        coeffs = self.wavelet_decompose(series)
        approx = coeffs[0]  # Use approximation coeffs as input to LSTM
        approx_series = TimeSeries.from_values(approx.reshape(-1, 1))
        
        # Fit the internal LSTM model
        self.lstm_model.fit(approx_series)

    def predict(self, n,series):
        pred = self.lstm_model.predict(n)
        return pred
    def save(self, path):
        os.makedirs(path, exist_ok=True)

        # Save LSTM model
        self.lstm_model.save(os.path.join(path, "lstm_model"))

        # Save wavelet settings and kwargs
        meta = {
            'wavelet': self.wavelet,
            'level': self.level,
            'lstm_kwargs': self.lstm_kwargs
        }
        with open(os.path.join(path, "meta.json"), "w") as f:
            json.dump(meta, f)

    @classmethod
    def load(cls, path):
        with open(os.path.join(path, "meta.json"), "r") as f:
            meta = json.load(f)
        instance = cls(**meta)
        instance.lstm_model = RNNModel.load(os.path.join(path, "lstm_model"))
        return instance






window_size = 12
num_epochs=300
model_constructors = {
    'LSTM'      : lambda: RNNModel(model='LSTM', input_chunk_length=window_size, output_chunk_length=6,
                                   hidden_dim=25, n_rnn_layers=2, dropout=0.1,
                                   batch_size=16, n_epochs=num_epochs, optimizer_kwargs={'lr':1e-3},
                                   random_state=SEED),
    'SVR'       : lambda: RegressionModel(model=SVR(),lags=window_size),
    'RandomRF'  : lambda: RandomForest(lags=window_size, n_estimators=100, random_state=SEED),
    'ExtraTF'   : lambda: XGBModel(lags=window_size, objective='reg:squarederror', random_state=SEED),
    'WBBLSTM'   : lambda: WBBLSTMModel(
        wavelet='db1',
        level=1,
        input_chunk_length=12,
        output_chunk_length=6,
        hidden_dim=25,
        n_rnn_layers=2,
        dropout=0.1,
        batch_size=16,
        n_epochs=num_epochs,
        optimizer_kwargs={'lr':1e-3},
        random_state=SEED
    )
}


def compute_metrics(obs: np.ndarray, pred: np.ndarray):
    rmse = np.sqrt(mean_squared_error(obs, pred))
    std_obs, std_pred = np.std(obs), np.std(pred)
    corr = np.corrcoef(obs, pred)[0,1]
    return std_obs, std_pred, corr, rmse

results = {}  
base_dir = "all_results"
os.makedirs(base_dir, exist_ok=True)



for station_id, group in all_spi.groupby('station_id'):
    results[station_id] = {}
    for col in [c for c in group.columns if c.startswith("SPI_")]:

        print(f"\n=== Station {station_id} | {col} ===")
        sub_df = group[['ds', col]].dropna()

        series = TimeSeries.from_dataframe(sub_df, time_col='ds', value_cols=col)
        
        # split once per series
        train, val = series.split_before(0.8)


        model_folder = os.path.join(base_dir, "{station_id}/{col}")

        os.makedirs(model_folder, exist_ok=True)


        model_stats = {}
        forecasts = {}
        for name, mk in model_constructors.items():
            model_file = os.path.join(model_folder, f"{name}.pkl")

            if os.path.exists(model_file):
                print(f"Model {name} already trained. Skipping.")
                # model = load(model_file)
                model_class = {
                    'LSTM': RNNModel,
                    'SVR': RegressionModel,
                    'RandomRF': RandomForest,
                    'ExtraTF': XGBModel,
                    'WBBLSTM': WBBLSTMModel
                }[name]
                model = model_class.load(model_file)

            else:
                print(f" Training {name}…", end='')
                model = mk()
                model.fit(train)
                model.save(model_file)

                print(" saved.")


            forecast = model.predict(len(val), series=train)
            o = val.values().flatten()
            p = forecast.values().flatten()
            std_o, std_p, corr, rmse = compute_metrics(o, p)
            model_stats[name] = (std_o, std_p, corr, rmse)
        
        results[station_id][col] = model_stats




Loop over stations & timescales, train & forecast

In [ ]:
from statsmodels.tsa.seasonal import STL
from sklearn.ensemble import IsolationForest
import os


base_dir = "all_results"
os.makedirs(base_dir, exist_ok=True)

for station_id, group in all_spi.groupby('station_id'):
        station_dir = os.path.join(base_dir, str(station_id))
        os.makedirs(station_dir, exist_ok=True)
        # for col in [c for c in group.columns if c.startswith("SPI_")]:

            # Taylor Diagram
            ref_std = list(model_stats.values())[0][0]
            fig = plt.figure(figsize=(6, 6))
            taylor = TaylorDiagram(ref_std, fig, label='Observed')
            for name, (std_o, std_p, corr, rmse) in model_stats.items():
                taylor.add_sample(std_p, corr, label=name)
            taylor.add_contours(levels=5, colors='0.5')
            plt.legend()
            plt.title(f"Taylor Diagram: Station {station_id} | {col}")
            plt.tight_layout()
            fig.savefig(f"{station_dir}/taylor_{col}.png")
            plt.close()

            # plt.show()

            # Best model
            best_model = min(model_stats.items(), key=lambda x: x[1][3])[0]
            print(f"Best model for {station_id} | {col}: {best_model}")
            # best = model_constructors[best_model]()
            # best.fit(series)
            model_file = os.path.join(model_folder, f"{best_model}.pkl")

            if os.path.exists(model_file):
                print(f"Model {name} already trained. Skipping.")
                # model = load(model_file)
                model_class = {
                    'LSTM': RNNModel,
                    'SVR': RegressionModel,
                    'RandomRF': RandomForest,
                    'ExtraTF': XGBModel,
                    'WBBLSTM': WBBLSTMModel
                }[name]
                best = model_class.load(model_file)

            # Forecast to 2050
            horizon = (pd.Timestamp("2050-12-01") - series.end_time()).days // 30
            future = best.predict(horizon)
            plot_spi_trend(series, future, title=f"SPI Forecast to 2050 — {station_id} | {col}")

            # Plot val vs forecast
            pred = forecasts[best_model]
            plt.figure(figsize=(12, 4))
            val.plot(label="Observed")
            pred.plot(label=f"Forecast - {best_model}")
            plt.title(f"Validation vs Prediction — {station_id} | {col}")
            plt.grid(True)
            plt.legend()
            plt.tight_layout()
            plt.show()

            # Prepare SPI dataframe for heatmap + category
            spi_series = sub_df.copy()
            spi_series['year'] = pd.to_datetime(spi_series['ds']).dt.year
            spi_series['month'] = pd.to_datetime(spi_series['ds']).dt.month

            # SPI heatmap
            heatmap_data = spi_series.pivot_table(index='year', columns='month', values=col)
            plt.figure(figsize=(12, 8))
            sns.heatmap(heatmap_data, cmap='rocket', center=0, annot=True, fmt=".2f")
            plt.title(f"SPI Heatmap — {col} — {station_id}")
            plt.xlabel("Month")
            plt.ylabel("Year")
            plt.tight_layout()
            plt.show()

            # Categorical SPI scatter
            spi_series['category'] = pd.cut(spi_series[col], bins=[-np.inf, -1, 1, np.inf], labels=['Dry', 'Normal', 'Wet'])
            colors = {'Dry': 'red', 'Normal': 'gray', 'Wet': 'blue'}

            plt.figure(figsize=(14, 6))
            plt.scatter(spi_series['ds'], spi_series[col], c=spi_series['category'].map(colors), label='SPI Category')
            plt.axhline(0, color='black', lw=1, linestyle='--')
            plt.title(f"SPI Categories: Dry / Normal / Wet — {station_id} | {col}")
            plt.xlabel("Date")
            plt.ylabel("SPI Value")
            plt.grid(True)
            plt.tight_layout()
            plt.legend()
            plt.show()

            print(f"✔️ Done with {station_id} | {col}\n")


Plotting a Taylor Diagram

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def taylor_diagram(std_ref, models_stats, title="Taylor Diagram"):
    """
    std_ref : float
        standard deviation of reference.
    models_stats : dict
        { model_name: (std_ref, std_model, corrcoef, rmse) }
    """
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111, polar=True)
    ax.set_theta_zero_location('E')
    ax.set_theta_direction(-1)
    
    # radial limits
    maxstd = max(v[1] for v in models_stats.values()) * 1.1
    ax.set_rmax(maxstd)
    ax.set_rticks(np.linspace(0, maxstd, 4))
    ax.set_rlabel_position(135)
    ax.grid(True)
    
    # plot reference point
    ax.plot(0, std_ref, 'k*', label='Reference')
    
    # plot each model
    for name, (std_o, std_p, corr, rmse) in models_stats.items():
        theta = np.arccos(corr)  # angle from zero
        ax.plot(theta, std_p, 'o', label=f"{name} (RMSE={rmse:.2f})")
    
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    ax.set_title(title)
    plt.tight_layout()
    plt.show()




for station_id, timescale_dict in results.items():
    for timescale, stats in timescale_dict.items():
        std_ref = next(iter(stats.values()))[0]  # same for every model
        title = f"Station {station_id} | {timescale}"
        taylor_diagram(std_ref, stats, title=title)


In [ ]:

from darts import TimeSeries
from darts.models import RNNModel
spi_series = all_spi_data[40708].dropna()

ts = TimeSeries.from_dataframe(spi_series, 'ds', 'SPI_3')

model = RNNModel(
    model='LSTM',
    input_chunk_length=12,    # e.g. use one year of past data
    output_chunk_length=6,    # forecast 6 months at a time
    hidden_dim=25,           # size of hidden layer (try adjusting)
    n_rnn_layers=2,
    dropout=0.1,
    batch_size=16,
    n_epochs=300,             # increase if needed for better convergence
    optimizer_kwargs={'lr': 1e-3},
    random_state=42
)

In [ ]:
model.fit(ts, verbose=True,)

In [ ]:
import pandas as pd

# End date of your time series:
last_date = ts.end_time().to_pydatetime()

# Calculate forecast horizon (number of months until January 2050).
# (Assuming you want to forecast up to and including December 2049,
#  so that the new data start from January 2050.)
forecast_steps = (2050 - last_date.year) * 12 - (last_date.month - 1)

print(f"Forecasting {forecast_steps} months into the future.")

import matplotlib.pyplot as plt

# Predict the future SPI until 2050.
forecast = model.predict(n=forecast_steps)

# Plot the original time series and the forecast.
plt.figure(figsize=(14, 7))
ts.plot(label='Observed SPI', lw=2)
forecast.plot(label='Forecast SPI', lw=2)

plt.title("SPI Forecast with LSTM (Darts) until 2050")
plt.xlabel("Date")
plt.ylabel("SPI Value")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming ts is a pandas Series with datetime index
# df = ts.to_frame(name='SPI')
spi_series = all_spi_data[40708].dropna()


spi_series['year'] = spi_series['ds'].dt.year
spi_series['month'] = spi_series['ds'].dt.month

# Pivot to heatmap form: each row = year, columns = month
heatmap_data = spi_series.pivot_table(index='year', columns='month', values='SPI_3')

plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, cmap='rocket', center=0, annot=True, fmt=".2f")
plt.title("SPI Heatmap: Dry (-) and Wet (+) Conditions by Month and Year")
plt.xlabel("Month")
plt.ylabel("Year")
plt.grid(False)

plt.show()


In [ ]:
spi_series['category'] = pd.cut(spi_series['SPI_3'], bins=[-np.inf, -1, 1, np.inf], labels=['Dry', 'Normal', 'Wet'])

colors = {'Dry': 'red', 'Normal': 'gray', 'Wet': 'blue'}

plt.figure(figsize=(14, 6))
plt.scatter(spi_series.index, spi_series['SPI_3'], c=spi_series['category'].map(colors), label='SPI by Category')
plt.axhline(0, color='black', lw=1, linestyle='--')
plt.title("SPI Categorized as Dry, Normal, and Wet Days")
plt.xlabel("Date")
plt.ylabel("SPI Value")
plt.grid(True)
plt.tight_layout()
plt.legend()
plt.show()
